<a href="https://colab.research.google.com/github/joshuadollison/smallbizpulse/blob/jd-model/notebooks/model_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
# ============================================================
# SETUP: Mount Drive, Install Dependencies, Configure Styling
# ============================================================

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install VADER for sentiment analysis
!pip install vaderSentiment -q

import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
from collections import Counter
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# ── Consistent Plot Styling ──────────────────────────────────
plt.rcParams.update({
    'figure.figsize': (12, 6),
    'figure.dpi': 120,
    'font.family': 'sans-serif',
    'font.size': 11,
    'axes.titlesize': 14,
    'axes.titleweight': 'bold',
    'axes.labelsize': 12,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': True,
    'grid.alpha': 0.3,
    'grid.linestyle': '--',
})

# SmallBizPulse color palette
COLORS = {
    'primary': '#2563EB',
    'secondary': '#F59E0B',
    'open': '#10B981',
    'closed': '#EF4444',
    'accent1': '#8B5CF6',
    'accent2': '#EC4899',
    'neutral': '#6B7280',
    'bg': '#F9FAFB',
}
PALETTE_OC = [COLORS['open'], COLORS['closed']]

print("Setup complete — libraries loaded, styling configured.")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 4.8 MB/s eta 0:00:00
Setup complete — libraries loaded, styling configured.


In [2]:
# ============================================================
# DATA LOADING
# ============================================================
# >>> UPDATE THIS PATH to match your Google Drive folder <<<
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/CIS509/yelp_dataset_new/'

def load_json(filename):
    filepath = DATA_PATH + filename
    with open(filepath, 'r') as f:
        first_char = f.read(1)
        f.seek(0)
        if first_char == '[':
            return pd.DataFrame(json.load(f))
        else:
            return pd.read_json(f, lines=True)

print("Loading datasets...")
business_df = load_json('yelp_academic_dataset_business.json')
print(f"  Business: {len(business_df):,} records")

review_df = load_json('yelp_academic_dataset_review.json')
print(f"  Review:   {len(review_df):,} records")

tip_df = load_json('yelp_academic_dataset_tip.json')
print(f"  Tip:      {len(tip_df):,} records")

checkin_df = load_json('yelp_academic_dataset_checkin.json')
print(f"  Checkin:  {len(checkin_df):,} records")

user_df = load_json('yelp_academic_dataset_user.json')
print(f"  User:     {len(user_df):,} records")

print("\nAll datasets loaded successfully.")

Loading datasets...
  Business: 9,973 records
  Review:   100,000 records
  Tip:      264,693 records
  Checkin:  9,337 records
  User:     79,345 records

All datasets loaded successfully.


In [3]:
# ============================================================
# DATA SOURCES & FILTERING CRITERIA
# ============================================================

# Step 1: Filter for restaurants
# A business is classified as a "restaurant" if its Yelp categories
# contain the word "Restaurants" (case-insensitive).
restaurant_df = business_df[
    business_df['categories'].str.contains('Restaurants', case=False, na=False)
].copy()

# Step 2: Get restaurant business IDs
restaurant_ids = set(restaurant_df['business_id'])

# Step 3: Filter reviews to restaurant-only
rest_review_df = review_df[review_df['business_id'].isin(restaurant_ids)].copy()
rest_review_df['date'] = pd.to_datetime(rest_review_df['date'])
rest_review_df['year'] = rest_review_df['date'].dt.year

# Step 4: Filter tips to restaurant-only
rest_tip_df = tip_df[tip_df['business_id'].isin(restaurant_ids)].copy()

# Step 5: Filter checkins to restaurant-only
rest_checkin_df = checkin_df[checkin_df['business_id'].isin(restaurant_ids)].copy()

# Step 6: Merge business status onto reviews
status_map = restaurant_df.set_index('business_id')['is_open'].to_dict()
rest_review_df['is_open'] = rest_review_df['business_id'].map(status_map)
rest_review_df['status'] = rest_review_df['is_open'].map({1: 'Open', 0: 'Closed'})

# ── Print Summary ────────────────────────────────────────────
print("=" * 65)
print("DATA SOURCES & FILTERING SUMMARY")
print("=" * 65)

print("\nPRIMARY DATA SOURCE: Yelp Academic Dataset")
print("-" * 45)

print("\nFull Dataset:")
print(f"  Businesses:  {len(business_df):>8,}")
print(f"  Reviews:     {len(review_df):>8,}")
print(f"  Tips:        {len(tip_df):>8,}")
print(f"  Check-ins:   {len(checkin_df):>8,}")
print(f"  Users:       {len(user_df):>8,}")

print("\nFiltered to Restaurants (categories contain 'Restaurants'):")
print(f"  Restaurants:        {len(restaurant_df):>8,}")
print(f"  Restaurant Reviews: {len(rest_review_df):>8,}")
print(f"  Restaurant Tips:    {len(rest_tip_df):>8,}")
print(f"  Restaurant Checkins:{len(rest_checkin_df):>8,}")

n_open = (restaurant_df['is_open'] == 1).sum()
n_closed = (restaurant_df['is_open'] == 0).sum()
print("\nRestaurant Status:")
print(f"  Open:   {n_open:>5,}  ({n_open / len(restaurant_df) * 100:.1f}%)")
print(f"  Closed: {n_closed:>5,}  ({n_closed / len(restaurant_df) * 100:.1f}%)")

print(f"\nDate Range: {rest_review_df['date'].min().strftime('%Y-%m-%d')} to "
      f"{rest_review_df['date'].max().strftime('%Y-%m-%d')}")

print("\nFILTERING CRITERIA APPLIED:")
print("  1. Category filter: categories.str.contains('Restaurants')")
print("  2. Reviews, tips, and check-ins filtered by restaurant business_id")
print("  3. No minimum review count threshold (preserving data-sparse")
print("     businesses is important for studying closure patterns)")
print("  4. No date range restriction (full temporal span needed for time-series)")

DATA SOURCES & FILTERING SUMMARY

PRIMARY DATA SOURCE: Yelp Academic Dataset
---------------------------------------------

Full Dataset:
  Businesses:     9,973
  Reviews:      100,000
  Tips:         264,693
  Check-ins:      9,337
  Users:         79,345

Filtered to Restaurants (categories contain 'Restaurants'):
  Restaurants:           4,132
  Restaurant Reviews:   72,124
  Restaurant Tips:      20,394
  Restaurant Checkins:   4,085

Restaurant Status:
  Open:   2,575  (62.3%)
  Closed: 1,557  (37.7%)

Date Range: 2005-03-01 to 2018-10-04

FILTERING CRITERIA APPLIED:
  1. Category filter: categories.str.contains('Restaurants')
  2. Reviews, tips, and check-ins filtered by restaurant business_id
  3. No minimum review count threshold (preserving data-sparse
     businesses is important for studying closure patterns)
  4. No date range restriction (full temporal span needed for time-series)


# Get some counts

- wanted to see counts per month to get a sense of the types of model and windows we would want for regression/time-series

In [4]:
import pandas as pd

# Safety - ensure datetime (EDA notebook already does this, but this won't hurt)
rest_review_df['date'] = pd.to_datetime(rest_review_df['date'], errors='coerce')

# 1) Overall monthly review counts
monthly_counts = (
    rest_review_df
      .dropna(subset=['date'])
      .groupby(rest_review_df['date'].dt.to_period('M'))
      .size()
      .rename('review_count')
      .reset_index(name='review_count')
      .rename(columns={'date': 'month'})
)

# Convert Period to timestamp for easy plotting/merging (month start)
monthly_counts['month'] = monthly_counts['month'].dt.to_timestamp()

print(monthly_counts.head(12))
print('\nRows:', len(monthly_counts))
print('Date range:', monthly_counts['month'].min(), 'to', monthly_counts['month'].max())

# 2) Monthly counts split by business status (Open vs Closed) - if you created 'status' in EDA
if 'status' in rest_review_df.columns:
    monthly_counts_by_status = (
        rest_review_df
          .dropna(subset=['date'])
          .groupby([rest_review_df['date'].dt.to_period('M'), 'status'])
          .size()
          .rename('review_count')
          .reset_index()
          .rename(columns={'date': 'month'})
    )
    monthly_counts_by_status['month'] = monthly_counts_by_status['month'].dt.to_timestamp()
    print('\nBy status:')
    print(monthly_counts_by_status.head(12))

# 3) Monthly counts per business_id (useful for later time-series modeling)
monthly_counts_by_business = (
    rest_review_df
      .dropna(subset=['date'])
      .groupby(['business_id', rest_review_df['date'].dt.to_period('M')])
      .size()
      .rename('review_count')
      .reset_index()
      .rename(columns={'date': 'month'})
)
monthly_counts_by_business['month'] = monthly_counts_by_business['month'].dt.to_timestamp()

print('\nPer business:')
print(monthly_counts_by_business.head(12))

        month  review_count
0  2005-03-01             4
1  2005-04-01             3
2  2005-05-01             4
3  2005-06-01             1
4  2005-07-01            14
5  2005-08-01             1
6  2005-09-01             7
7  2005-10-01             2
8  2005-11-01             7
9  2005-12-01             4
10 2006-01-01            13
11 2006-02-01             5

Rows: 156
Date range: 2005-03-01 00:00:00 to 2018-10-01 00:00:00

By status:
        month  status  review_count
0  2005-03-01  Closed             1
1  2005-03-01    Open             3
2  2005-04-01  Closed             2
3  2005-04-01    Open             1
4  2005-05-01  Closed             2
5  2005-05-01    Open             2
6  2005-06-01    Open             1
7  2005-07-01  Closed             7
8  2005-07-01    Open             7
9  2005-08-01    Open             1
10 2005-09-01  Closed             3
11 2005-09-01    Open             4

Per business:
               business_id      month  review_count
0   --ZVrH2X2QXBFdCilbi

In [5]:
# Pivot for quick plot-ready table (month rows, status columns)
if 'status' in rest_review_df.columns:
    pivot = monthly_counts_by_status.pivot(index='month', columns='status', values='review_count').fillna(0).astype(int)
    print(pivot.tail(12))


status      Closed  Open
month                   
2017-11-01     129   529
2017-12-01     181   904
2018-01-01     141   652
2018-02-01     202   888
2018-03-01     220  1035
2018-04-01     202  1012
2018-05-01     177   924
2018-06-01     144   844
2018-07-01     182   967
2018-08-01     137   799
2018-09-01     105   749
2018-10-01       8    38


In [6]:
import pandas as pd

# Ensure datetime
rest_review_df['date'] = pd.to_datetime(rest_review_df['date'], errors='coerce')
df = rest_review_df.dropna(subset=['date']).copy()

# Ensure status exists (Open/Closed)
if 'status' not in df.columns:
    if 'is_open' in df.columns:
        df['status'] = df['is_open'].map({1: 'Open', 0: 'Closed'})
    else:
        raise ValueError("Need either 'status' or 'is_open' in rest_review_df.")

# Build year-month grain counts (so the averaging is fair across years)
df['year'] = df['date'].dt.year
df['month_num'] = df['date'].dt.month
df['month_name'] = df['date'].dt.strftime('%b')  # Jan, Feb, ...

monthly_counts = (
    df.groupby(['status', 'year', 'month_num', 'month_name'])
      .size()
      .reset_index(name='review_count')
)

# Average by calendar month across years
avg_by_month = (
    monthly_counts.groupby(['status', 'month_num', 'month_name'])['review_count']
      .mean()
      .reset_index(name='avg_reviews_per_month')
      .sort_values(['month_num', 'status'])
)

# Nice pivot view (rows = month, cols = status)
avg_by_month_pivot = (
    avg_by_month.pivot(index=['month_num', 'month_name'], columns='status', values='avg_reviews_per_month')
      .reset_index()
      .sort_values('month_num')
)

print(avg_by_month_pivot)

status  month_num month_name      Closed        Open
0               1        Jan  133.230769  343.076923
1               2        Feb  159.300000  356.083333
2               3        Mar  148.333333  392.833333
3               4        Apr  121.538462  348.230769
4               5        May  131.000000  358.285714
5               6        Jun  118.307692  318.000000
6               7        Jul  140.615385  404.461538
7               8        Aug  142.692308  367.571429
8               9        Sep  104.923077  320.833333
9              10        Oct  135.692308  342.153846
10             11        Nov  101.833333  273.000000
11             12        Dec  119.000000  298.166667


# Build the Business-Month Feature Table

We start by creating a clean monthly view of the reviews dataset.  First, we convert each review timestamp into a month bucket (YYYY-MM) so we can measure activity and behavior at a consistent time grain.  

We then compute:
1. Total monthly review volume split by business status (Open vs Closed)
2. A business-month feature table that aggregates review behavior for each restaurant each month (review count, average star rating, rating mix, engagement signals, and basic text length statistics).  

This business-month table becomes the backbone for the modeling pipeline - we will later enrich it with neural-network sentiment scores and BERTopic topic proportions, then feed sequences of monthly features into a GRU/RNN to predict future sentiment direction and closure risk.

In [7]:
import pandas as pd

df = rest_review_df.copy()

# Ensure datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date'])

# Month bucket (month start timestamp)
df['month'] = df['date'].dt.to_period('M').dt.to_timestamp()

# 1) Monthly totals by status (Open vs Closed)
monthly_by_status = (
    df.groupby(['status', 'month'])
      .size()
      .reset_index(name='review_count')
      .sort_values(['month', 'status'])
)

print(monthly_by_status.head(24))

# 2) Business-month backbone table (this is what the RNN will consume)
biz_month = (
    df.groupby(['business_id', 'status', 'month'])
      .agg(
          review_count=('review_id', 'count'),
          avg_stars=('stars', 'mean'),
          pct_1star=('stars', lambda s: (s <= 1.0).mean()),
          pct_5star=('stars', lambda s: (s >= 5.0).mean()),
          avg_useful=('useful', 'mean'),
          avg_funny=('funny', 'mean'),
          avg_cool=('cool', 'mean'),
          avg_text_len=('text', lambda x: x.fillna('').str.len().mean()),
          avg_word_count=('text', lambda x: x.fillna('').str.split().str.len().mean()),
      )
      .reset_index()
      .sort_values(['business_id', 'month'])
)

print(biz_month.head(20))

# 3) Optional: filter to businesses with enough activity for monthly sequences
# Example rule: at least 12 total business-month rows in the dataset
eligible = (
    biz_month.groupby('business_id')['month']
             .nunique()
             .reset_index(name='n_months')
)
eligible_ids = eligible.loc[eligible['n_months'] >= 12, 'business_id']

biz_month_eligible = biz_month[biz_month['business_id'].isin(eligible_ids)].copy()
print("Eligible businesses:", biz_month_eligible['business_id'].nunique())
print("Eligible rows:", len(biz_month_eligible))

     status      month  review_count
0    Closed 2005-03-01             1
150    Open 2005-03-01             3
1    Closed 2005-04-01             2
151    Open 2005-04-01             1
2    Closed 2005-05-01             2
152    Open 2005-05-01             2
153    Open 2005-06-01             1
3    Closed 2005-07-01             7
154    Open 2005-07-01             7
155    Open 2005-08-01             1
4    Closed 2005-09-01             3
156    Open 2005-09-01             4
5    Closed 2005-10-01             1
157    Open 2005-10-01             1
6    Closed 2005-11-01             2
158    Open 2005-11-01             5
7    Closed 2005-12-01             2
159    Open 2005-12-01             2
8    Closed 2006-01-01             7
160    Open 2006-01-01             6
9    Closed 2006-02-01             2
161    Open 2006-02-01             3
10   Closed 2006-03-01             4
162    Open 2006-03-01             6
               business_id  status      month  review_count  avg_stars  \
0

# VADER comparison

## 1) Create a baseline sentiment score (VADER) per review

This gives you an immediate, cheap sentiment channel to compare against the NN later.

In [8]:
import numpy as np
import pandas as pd

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

df = rest_review_df.copy()
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date'])

analyzer = SentimentIntensityAnalyzer()

# VADER scores
vader = df['text'].fillna('').apply(analyzer.polarity_scores)
df['vader_neg'] = vader.apply(lambda d: d['neg'])
df['vader_neu'] = vader.apply(lambda d: d['neu'])
df['vader_pos'] = vader.apply(lambda d: d['pos'])
df['vader_compound'] = vader.apply(lambda d: d['compound'])

df[['review_id','stars','vader_compound']].head()

,review_id,stars,vader_compound
0,KU_O5udG6zpxOg-VcAEodg,3.0,0.8597
2,saUsX_uimxRlCVr67Z4Jig,3.0,0.9201
3,AqPFMleE6RsU23_auESxiA,5.0,0.9588
4,Sx8TMOWLNuJBWer-0pcmoA,4.0,0.9815
5,JrIxlS1TzJ-iCu79ul40cQ,1.0,0.7117


## 2) Aggregate VADER to business-month features

This becomes part of the sequence input.

In [9]:
df['month'] = df['date'].dt.to_period('M').dt.to_timestamp()

biz_month_vader = (
    df.groupby(['business_id', 'status', 'month'])
      .agg(
          review_count=('review_id', 'count'),
          avg_stars=('stars', 'mean'),
          vader_mean=('vader_compound', 'mean'),
          vader_std=('vader_compound', 'std'),
          neg_share=('vader_compound', lambda s: (s < -0.05).mean()),
          pos_share=('vader_compound', lambda s: (s >  0.05).mean()),
      )
      .reset_index()
      .sort_values(['business_id', 'month'])
)

biz_month_vader.head(20)

,business_id,status,month,review_count,avg_stars,vader_mean,vader_std,neg_share,pos_share
0,--ZVrH2X2QXBFdCilbirsw,Closed,2013-07-01,1,5.0,0.8856,NaN,0.0,1.0
1,--ZVrH2X2QXBFdCilbirsw,Closed,2014-03-01,1,5.0,0.7777,NaN,0.0,1.0
2,--ZVrH2X2QXBFdCilbirsw,Closed,2014-12-01,1,5.0,0.8646,NaN,0.0,1.0
3,--ZVrH2X2QXBFdCilbirsw,Closed,2015-02-01,1,3.0,0.8921,NaN,0.0,1.0
4,--ZVrH2X2QXBFdCilbirsw,Closed,2015-05-01,1,5.0,0.6468,NaN,0.0,1.0
5,--ZVrH2X2QXBFdCilbirsw,Closed,2016-02-01,2,5.0,0.8821,0.076085,0.0,1.0
6,--ZVrH2X2QXBFdCilbirsw,Closed,2016-03-01,1,5.0,0.9449,NaN,0.0,1.0
7,--ZVrH2X2QXBFdCilbirsw,Closed,2017-07-01,1,5.0,0.9794,NaN,0.0,1.0
8,--ZVrH2X2QXBFdCilbirsw,Closed,2018-02-01,1,5.0,0.0000,NaN,0.0,0.0
9,-1MhPXk1FglglUAmuPLIGg,Open,2009-03-01,1,3.0,0.8020,NaN,0.0,1.0


# Sentiment Analysis

In this section we fine-tune a transformer-based sentiment model on our own Yelp review data to produce a high-quality, domain-specific sentiment signal.  We create a supervised training set using clearly polarized reviews (1-star = negative, 5-star = positive), split it into train/validation sets, and fine-tune DistilBERT to classify review sentiment.  After training, we use the best checkpoint to score every review with a continuous probability of positive sentiment (0-1).  Finally, we aggregate these transformer sentiment scores to the business-month level (mean, variability, and positive/negative share), creating time-series features that will later feed our GRU model for forecasting sentiment trajectories and predicting closure risk.

In [13]:
!pip -q install -U transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 51.6 MB/s eta 0:00:00


In [18]:
import os
import math
import random
import numpy as np
import pandas as pd

# ----------------------------
# 0) Reproducibility
# ----------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# ----------------------------
# 1) Prep data
# ----------------------------
df = rest_review_df.copy()
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date']).copy()
df['text'] = df['text'].fillna('').astype(str)

# Fine-tune labels: 1-star = 0 (negative), 5-star = 1 (positive)
train_df = df[df['stars'].isin([1.0, 5.0])].copy()
train_df['label'] = (train_df['stars'] == 5.0).astype(int)

print("Fine-tune rows:", len(train_df))
print(train_df['label'].value_counts())

# ----------------------------
# 2) Stratified train/val split (no sklearn needed)
# ----------------------------
y = train_df['label'].values
idx_pos = np.where(y == 1)[0]
idx_neg = np.where(y == 0)[0]
np.random.shuffle(idx_pos)
np.random.shuffle(idx_neg)

split_pos = int(0.8 * len(idx_pos))
split_neg = int(0.8 * len(idx_neg))

tr_idx = np.concatenate([idx_pos[:split_pos], idx_neg[:split_neg]])
va_idx = np.concatenate([idx_pos[split_pos:], idx_neg[split_neg:]])
np.random.shuffle(tr_idx)
np.random.shuffle(va_idx)

train_split = train_df.iloc[tr_idx].reset_index(drop=True)
val_split   = train_df.iloc[va_idx].reset_index(drop=True)

print("Train split:", len(train_split), "Val split:", len(val_split))
print("Train label dist:\n", train_split['label'].value_counts(normalize=True))
print("Val label dist:\n", val_split['label'].value_counts(normalize=True))

# ----------------------------
# 3) Transformers fine-tune (DistilBERT)
# ----------------------------
# Install if needed:
# pip install -U transformers datasets accelerate torch

import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)

MODEL_NAME = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Keep max_len reasonable - Yelp reviews can be long.
MAX_LEN = 256

def tokenize_batch(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_LEN
    )

train_ds = Dataset.from_pandas(train_split[['text', 'label']])
val_ds   = Dataset.from_pandas(val_split[['text', 'label']])

train_ds = train_ds.map(tokenize_batch, batched=True, remove_columns=["text"])
val_ds   = val_ds.map(tokenize_batch, batched=True, remove_columns=["text"])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    # confusion components
    tp = int(((preds == 1) & (labels == 1)).sum())
    tn = int(((preds == 0) & (labels == 0)).sum())
    fp = int(((preds == 1) & (labels == 0)).sum())
    fn = int(((preds == 0) & (labels == 1)).sum())

    acc = (tp + tn) / max(1, tp + tn + fp + fn)
    precision = tp / max(1, tp + fp)
    recall = tp / max(1, tp + fn)
    f1 = 2 * precision * recall / max(1e-12, (precision + recall))

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

# Output dir in repo artifacts
from transformers import TrainingArguments

OUT_DIR = "../artifacts/transformer_sentiment_distilbert"

steps_per_epoch = math.ceil(len(train_ds) / (16 * 2))
total_steps = steps_per_epoch * 2
warmup_steps = int(0.06 * total_steps)

training_args = TrainingArguments(
    output_dir=OUT_DIR,
    seed=SEED,

    # v5 uses eval_strategy (NOT evaluation_strategy)
    eval_strategy="epoch",
    save_strategy="epoch",

    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,

    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,
    warmup_steps=warmup_steps,
    weight_decay=0.01,
    logging_steps=100,

    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()
eval_metrics = trainer.evaluate()
print("Eval metrics:", eval_metrics)

# Save the best model + tokenizer
trainer.save_model(OUT_DIR)
tokenizer.save_pretrained(OUT_DIR)
print("Saved to:", OUT_DIR)

# ----------------------------
# 4) Score ALL reviews with the fine-tuned model (fast batch logits, no pipeline)
# ----------------------------
import numpy as np
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding

# Reload from saved directory (guarantees you're using the best checkpoint)
ft_model = AutoModelForSequenceClassification.from_pretrained(OUT_DIR)
ft_tokenizer = AutoTokenizer.from_pretrained(OUT_DIR)

# Device
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
ft_model.to(device)
ft_model.eval()

# Collator handles dynamic padding per batch
collator = DataCollatorWithPadding(tokenizer=ft_tokenizer, return_tensors="pt")

texts = df['text'].fillna('').astype(str).tolist()
BATCH_SIZE = 64

# We will create tx_sent in [0,1] = P(positive)
p_pos = []

with torch.no_grad():
    for i in range(0, len(texts), BATCH_SIZE):
        batch_texts = texts[i:i + BATCH_SIZE]

        # Tokenize this batch (no padding yet)
        enc = ft_tokenizer(
            batch_texts,
            truncation=True,
            max_length=MAX_LEN
        )

        # Convert to list-of-features for collator, then pad to tensors
        features = [{k: enc[k][j] for k in enc.keys()} for j in range(len(batch_texts))]
        batch = collator(features)

        # Move to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass -> logits -> softmax -> P(class=1)
        logits = ft_model(**batch).logits
        probs = torch.softmax(logits, dim=1)[:, 1].detach().cpu().numpy()

        p_pos.extend(probs.tolist())

df['tx_sent'] = np.array(p_pos, dtype=float)

print(df[['review_id', 'stars', 'tx_sent']].head(10))
print("tx_sent range:", df['tx_sent'].min(), "to", df['tx_sent'].max())

# ----------------------------
# 5) Aggregate to business-month features (feeds GRU time-series later)
# ----------------------------
df['month'] = df['date'].dt.to_period('M').dt.to_timestamp()

biz_month_tx = (
    df.groupby(['business_id', 'status', 'month'])
      .agg(
          review_count=('review_id', 'count'),
          avg_stars=('stars', 'mean'),
          tx_sent_mean=('tx_sent', 'mean'),
          tx_sent_std=('tx_sent', 'std'),
          tx_neg_share=('tx_sent', lambda s: (s < 0.40).mean()),
          tx_pos_share=('tx_sent', lambda s: (s > 0.60).mean()),
      )
      .reset_index()
      .sort_values(['business_id', 'month'])
)

print(biz_month_tx.head(20))
print("Rows:", len(biz_month_tx), "Businesses:", biz_month_tx['business_id'].nunique())

Fine-tune rows: 36897
label
1    29709
0     7188
Name: count, dtype: int64
Train split: 29517 Val split: 7380
Train label dist:
 label
1    0.805197
0    0.194803
Name: proportion, dtype: float64
Val label dist:
 label
1    0.805149
0    0.194851
Name: proportion, dtype: float64


Map:   0%|          | 0/29517 [00:00<?, ? examples/s]

Map:   0%|          | 0/7380 [00:00<?, ? examples/s]

Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_transform.weight  | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
classifier.bias         | MISSING    | 
pre_classifier.weight   | MISSING    | 
classifier.weight       | MISSING    | 
pre_classifier.bias     | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


...

Eval metrics: {'eval_loss': 0.050367340445518494, 'eval_accuracy': 0.9879403794037941, 'eval_precision': 0.991600873509155, 'eval_recall': 0.9934365533490407, 'eval_f1': 0.9925178646490123, 'eval_runtime': 14.8199, 'eval_samples_per_second': 497.978, 'eval_steps_per_second': 15.587, 'epoch': 2.0}


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Saved to: ../artifacts/transformer_sentiment_distilbert


Loading weights:   0%|          | 0/104 [00:00<?, ?it/s]

                 review_id  stars   tx_sent
0   KU_O5udG6zpxOg-VcAEodg    3.0  0.996110
2   saUsX_uimxRlCVr67Z4Jig    3.0  0.999877
3   AqPFMleE6RsU23_auESxiA    5.0  0.999888
4   Sx8TMOWLNuJBWer-0pcmoA    4.0  0.999741
5   JrIxlS1TzJ-iCu79ul40cQ    1.0  0.003565
7   _ZeMknuYdlQcUqng_Im3yg    5.0  0.999888
9   pUycOfUwM8vqX7KjRRhUEA    3.0  0.001226
11  l3Wk_mvAog6XANIuGQ9C7Q    4.0  0.999861
12  XW_LfMv0fV21l9c6xQd_lw    4.0  0.999876
13  8JFGBuHMoiNDyfcxuWNtrA    4.0  0.999824
tx_sent range: 0.0006120343459770083 to 0.9998959302902222
               business_id  status      month  review_count  avg_stars  \
0   --ZVrH2X2QXBFdCilbirsw  Closed 2013-07-01             1        5.0   
1   --ZVrH2X2QXBFdCilbirsw  Closed 2014-03-01             1        5.0   
2   --ZVrH2X2QXBFdCilbirsw  Closed 2014-12-01             1        5.0   
3   --ZVrH2X2QXBFdCilbirsw  Closed 2015-02-01             1        3.0   
4   --ZVrH2X2QXBFdCilbirsw  Closed 2015-05-01             1        5.0   
5   --ZVrH2X2

In [20]:
# ============================
# Cell 0 - Setup + Imports
# ============================
import os
import math
import random
import numpy as np
import pandas as pd

import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)

# ============================
# Cell 1 - Reproducibility
# ============================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print("CUDA available:", torch.cuda.is_available())

# ============================
# Cell 2 - Prep Data
# Assumes rest_review_df is already loaded like your EDA notebook.
# ============================
df = rest_review_df.copy()

df["date"] = pd.to_datetime(df["date"], errors="coerce")
df = df.dropna(subset=["date"]).copy()

df["text"] = df["text"].fillna("").astype(str)

# Fine-tune labels: 1-star = 0 (negative), 5-star = 1 (positive)
train_df = df[df["stars"].isin([1.0, 5.0])].copy()
train_df["label"] = (train_df["stars"] == 5.0).astype(int)

print("Fine-tune rows:", len(train_df))
print(train_df["label"].value_counts())

# ============================
# Cell 3 - Stratified Train/Val Split (no sklearn)
# ============================
y = train_df["label"].values

idx_pos = np.where(y == 1)[0]
idx_neg = np.where(y == 0)[0]

np.random.shuffle(idx_pos)
np.random.shuffle(idx_neg)

split_pos = int(0.8 * len(idx_pos))
split_neg = int(0.8 * len(idx_neg))

tr_idx = np.concatenate([idx_pos[:split_pos], idx_neg[:split_neg]])
va_idx = np.concatenate([idx_pos[split_pos:], idx_neg[split_neg:]])

np.random.shuffle(tr_idx)
np.random.shuffle(va_idx)

train_split = train_df.iloc[tr_idx].reset_index(drop=True)
val_split = train_df.iloc[va_idx].reset_index(drop=True)

print("Train split:", len(train_split), "Val split:", len(val_split))
print("Train label dist:\n", train_split["label"].value_counts(normalize=True))
print("Val label dist:\n", val_split["label"].value_counts(normalize=True))

# ============================
# Cell 4 - Tokenize + Build HF Datasets
# ============================
MODEL_NAME = "distilbert-base-uncased"
MAX_LEN = 256

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_batch(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_LEN
    )

train_ds = Dataset.from_pandas(train_split[["text", "label"]])
val_ds = Dataset.from_pandas(val_split[["text", "label"]])

train_ds = train_ds.map(tokenize_batch, batched=True, remove_columns=["text"])
val_ds = val_ds.map(tokenize_batch, batched=True, remove_columns=["text"])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ============================
# Cell 5 - Model + Metrics
# ============================
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    tp = int(((preds == 1) & (labels == 1)).sum())
    tn = int(((preds == 0) & (labels == 0)).sum())
    fp = int(((preds == 1) & (labels == 0)).sum())
    fn = int(((preds == 0) & (labels == 1)).sum())

    acc = (tp + tn) / max(1, tp + tn + fp + fn)
    precision = tp / max(1, tp + fp)
    recall = tp / max(1, tp + fn)
    f1 = 2 * precision * recall / max(1e-12, (precision + recall))

    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ============================
# Cell 6 - TrainingArguments (Transformers v5-safe) + Trainer + Train
# ============================
OUT_DIR = "../artifacts/transformer_sentiment_distilbert"

EPOCHS = 2
PER_DEVICE_TRAIN_BS = 16
PER_DEVICE_EVAL_BS = 32
GRAD_ACCUM = 2

# warmup_ratio is deprecated in v5.2 - use warmup_steps now.
steps_per_epoch = math.ceil(len(train_ds) / (PER_DEVICE_TRAIN_BS * GRAD_ACCUM))
total_steps = steps_per_epoch * EPOCHS
warmup_steps = int(0.06 * total_steps)

training_args = TrainingArguments(
    output_dir=OUT_DIR,
    seed=SEED,

    # Transformers v5 uses eval_strategy (NOT evaluation_strategy)
    eval_strategy="epoch",
    save_strategy="epoch",

    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,

    num_train_epochs=EPOCHS,
    learning_rate=2e-5,
    per_device_train_batch_size=PER_DEVICE_TRAIN_BS,
    per_device_eval_batch_size=PER_DEVICE_EVAL_BS,
    gradient_accumulation_steps=GRAD_ACCUM,

    warmup_steps=warmup_steps,
    weight_decay=0.01,

    logging_steps=100,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

# Transformers v5 Trainer does NOT accept tokenizer=.  Use processing_class=.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
eval_metrics = trainer.evaluate()
print("Eval metrics:", eval_metrics)

trainer.save_model(OUT_DIR)
tokenizer.save_pretrained(OUT_DIR)
print("Saved to:", OUT_DIR)

# ============================
# Cell 7 - Temperature scaling (calibration) on validation set
# Output: best_T
# ============================
pred_out = trainer.predict(val_ds)
val_logits = torch.tensor(pred_out.predictions, dtype=torch.float32)
val_labels = torch.tensor(pred_out.label_ids, dtype=torch.long)

def nll_for_T(T: float) -> float:
    scaled = val_logits / T
    probs = torch.softmax(scaled, dim=1)
    p = probs[torch.arange(len(val_labels)), val_labels]
    return (-torch.log(p.clamp_min(1e-12))).mean().item()

Ts = np.linspace(0.5, 5.0, 46)  # 0.5, 0.6, ..., 5.0
losses = [nll_for_T(float(T)) for T in Ts]
best_T = float(Ts[int(np.argmin(losses))])

print("Best temperature:", best_T)
print("NLL @ best_T:", min(losses))

# ============================
# Cell 8 - Score ALL Reviews (fast batch logits, temperature-scaled)
# Output: df['tx_sent'] = calibrated P(positive), 0..1
# ============================
from transformers import DataCollatorWithPadding

ft_model = AutoModelForSequenceClassification.from_pretrained(OUT_DIR)
ft_tokenizer = AutoTokenizer.from_pretrained(OUT_DIR)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
ft_model.to(device)
ft_model.eval()

collator = DataCollatorWithPadding(tokenizer=ft_tokenizer, return_tensors="pt")

texts = df["text"].fillna("").astype(str).tolist()
BATCH_SIZE = 64

p_pos = []

with torch.no_grad():
    for i in range(0, len(texts), BATCH_SIZE):
        batch_texts = texts[i:i + BATCH_SIZE]

        enc = ft_tokenizer(
            batch_texts,
            truncation=True,
            max_length=MAX_LEN
        )

        features = [{k: enc[k][j] for k in enc.keys()} for j in range(len(batch_texts))]
        batch = collator(features)
        batch = {k: v.to(device) for k, v in batch.items()}

        logits = ft_model(**batch).logits
        probs = torch.softmax(logits / best_T, dim=1)[:, 1].detach().cpu().numpy()
        p_pos.extend(probs.tolist())

df["tx_sent"] = np.array(p_pos, dtype=float)

print(df[["review_id", "stars", "tx_sent"]].head(10))
print("tx_sent range:", df["tx_sent"].min(), "to", df["tx_sent"].max())

# ============================
# Cell 9 - Aggregate to Business-Month Features (feeds GRU time-series later)
# Output: biz_month_tx
# ============================
df["month"] = df["date"].dt.to_period("M").dt.to_timestamp()

biz_month_tx = (
    df.groupby(["business_id", "status", "month"])
      .agg(
          review_count=("review_id", "count"),
          avg_stars=("stars", "mean"),
          tx_sent_mean=("tx_sent", "mean"),
          tx_sent_std=("tx_sent", "std"),
          tx_neg_share=("tx_sent", lambda s: (s < 0.30).mean()),
          tx_pos_share=("tx_sent", lambda s: (s > 0.70).mean()),
      )
      .reset_index()
      .sort_values(["business_id", "month"])
)

# std is NaN when review_count == 1 - make it numeric for modeling
biz_month_tx["tx_sent_std"] = biz_month_tx["tx_sent_std"].fillna(0.0)

print(biz_month_tx.head(20))
print("Rows:", len(biz_month_tx), "Businesses:", biz_month_tx["business_id"].nunique())

CUDA available: True
Fine-tune rows: 36897
label
1    29709
0     7188
Name: count, dtype: int64
Train split: 29517 Val split: 7380
Train label dist:
 label
1    0.805197
0    0.194803
Name: proportion, dtype: float64
Val label dist:
 label
1    0.805149
0    0.194851
Name: proportion, dtype: float64


Map:   0%|          | 0/29517 [00:00<?, ? examples/s]

Map:   0%|          | 0/7380 [00:00<?, ? examples/s]

Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_transform.weight  | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
classifier.bias         | MISSING    | 
pre_classifier.weight   | MISSING    | 
classifier.weight       | MISSING    | 
pre_classifier.bias     | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


...

Eval metrics: {'eval_loss': 0.052605222910642624, 'eval_accuracy': 0.9872628726287263, 'eval_precision': 0.9910984212294256, 'eval_recall': 0.9930999663412993, 'eval_f1': 0.9920981842636181, 'eval_runtime': 14.8776, 'eval_samples_per_second': 496.047, 'eval_steps_per_second': 15.527, 'epoch': 2.0}


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Saved to: ../artifacts/transformer_sentiment_distilbert
Best temperature: 1.6
NLL @ best_T: 0.04107339307665825


Loading weights:   0%|          | 0/104 [00:00<?, ?it/s]

                 review_id  stars   tx_sent
0   KU_O5udG6zpxOg-VcAEodg    3.0  0.980491
2   saUsX_uimxRlCVr67Z4Jig    3.0  0.996636
3   AqPFMleE6RsU23_auESxiA    5.0  0.996777
4   Sx8TMOWLNuJBWer-0pcmoA    4.0  0.994773
5   JrIxlS1TzJ-iCu79ul40cQ    1.0  0.023795
7   _ZeMknuYdlQcUqng_Im3yg    5.0  0.996670
9   pUycOfUwM8vqX7KjRRhUEA    3.0  0.019927
11  l3Wk_mvAog6XANIuGQ9C7Q    4.0  0.996429
12  XW_LfMv0fV21l9c6xQd_lw    4.0  0.996370
13  8JFGBuHMoiNDyfcxuWNtrA    4.0  0.994894
tx_sent range: 0.008318239822983742 to 0.9969172477722168
               business_id  status      month  review_count  avg_stars  \
0   --ZVrH2X2QXBFdCilbirsw  Closed 2013-07-01             1        5.0   
1   --ZVrH2X2QXBFdCilbirsw  Closed 2014-03-01             1        5.0   
2   --ZVrH2X2QXBFdCilbirsw  Closed 2014-12-01             1        5.0   
3   --ZVrH2X2QXBFdCilbirsw  Closed 2015-02-01             1        3.0   
4   --ZVrH2X2QXBFdCilbirsw  Closed 2015-05-01             1        5.0   
5   --ZVrH2X2Q